BerkeleyX: Data8.3x

Foundations of Data Science: Prediction and Machine Learning

Section 7: Regression Inference

In [ ]:
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Lec 7.1 Regression Model

In [ ]:
def standard_units(any_numbers):
    """Convert any array of numbers to standard units."""
    return (any_numbers - np.average(any_numbers)) / np.std(any_numbers)

def correlation(t, x, y):
    """Return the correlation coefficient (r) of two variables."""
    return np.mean(standard_units(t.column(x)) * standard_units(t.column(y)))

def slope(t, x, y):
    """The slope of the regression line (original units)."""
    r = correlation(t, x, y)
    return r * np.std(t.column(y)) / np.std(t.column(x))

def intercept(t, x, y):
    """The intercept of the regression line (original units)."""
    return np.mean(t.column(y)) - slope(t, x, y) * np.mean(t.column(x))

def fit(t, x, y):
    """The fitted values along the regression line."""
    a = slope(t, x, y)
    b = intercept(t, x, y)
    return a * t.column(x) + b

def plot_residuals(t, x, y):
    """Plot a scatter diagram and residuals."""
    t.scatter(x, y, fit_line=True)
    actual = t.column(y)
    fitted = fit(t, x, y)
    residuals = actual - fitted
    print('r:', correlation(t, x, y))
    print('RMSE:', np.mean(residuals**2)**0.5)
    t.select(x).with_column('Residual', residuals).scatter(0, 1)

In [ ]:
def draw_and_compare(true_slope, true_int, sample_size):
    x = np.random.normal(50, 5, sample_size)
    xlims = np.array([np.min(x), np.max(x)])
    errors = np.random.normal(0, 6, sample_size)
    y = (true_slope * x + true_int) + errors
    sample = Table().with_columns('x', x, 'y', y)
    
    sample.scatter('x', 'y')
    plt.plot(xlims, true_slope*xlims + true_int, lw=2, color='green')
    plt.title('True Line, and Points Created')
    
    sample.scatter('x', 'y')
    plt.title('What We Get to See')
    
    sample.scatter('x', 'y', fit_line=True)
    plt.title('Regression Line: Estimate of True Line')
    
    sample.scatter('x', 'y', fit_line=True)
    plt.plot(xlims, true_slope*xlims + true_int, lw=2, color='green')
    plt.title('Regression Line and True Line')
    
draw_and_compare(2, -5, 10)

In [ ]:
draw_and_compare(2, -5, 100)

In [ ]:
draw_and_compare(2, -5, 1000)

Lec 7.2 Prediction Variability

In [ ]:
# looks like a linear correlation, at least in the central area
baby = Table.read_table('../../data/baby.csv')
baby.show(3)
plot_residuals(baby, 'Gestational Days', 'Birth Weight')

In [ ]:
# let's take a point somewhere in the middle
x = 300
a = slope(baby, 'Gestational Days', 'Birth Weight')
b = intercept(baby, 'Gestational Days', 'Birth Weight')
predicted_y = a * x + b
baby.scatter('Gestational Days', 'Birth Weight', fit_line=True)
print(predicted_y)
plt.scatter(300, predicted_y, color='gold', s=200);

In [ ]:
def prediction_at(t, x):
    a = slope(t, 'Gestational Days', 'Birth Weight')
    b = intercept(t, 'Gestational Days', 'Birth Weight')
    return a * x + b

# resampling, we always get about the same value
for i in range(4):
    resample = baby.sample()
    predicted_y = prediction_at(resample, 300)
    resample.scatter('Gestational Days', 'Birth Weight', fit_line=True)
    plt.scatter(300, predicted_y, color='gold', s=200)

In [ ]:
lines = Table(['slope', 'intercept', 'at 150', 'at 300', 'at 350'])

# let's draw 10 different regression lines
for i in range(10):
    resample = baby.sample()
    a = slope(resample, 'Gestational Days', 'Birth Weight')
    b = intercept(resample, 'Gestational Days', 'Birth Weight')
    lines.append([a, b, a * 150 + b, a * 300 + b, a * 350 + b])

# on the edges the lines diverge, caused by lack of information
baby.scatter('Gestational Days', 'Birth Weight')
for i in range(lines.num_rows):
    line = lines.row(i)
    plt.plot([150, 350], [line.item('at 150'), line.item('at 350')], lw=1)
    plt.scatter(300, line.item('at 300'), s=200)

In [ ]:
# zoom on the are we care more
lines = Table(['slope', 'intercept', 'at 291', 'at 300', 'at 309'])

for i in range(10):
    resample = baby.sample()
    a = slope(resample, 'Gestational Days', 'Birth Weight')
    b = intercept(resample, 'Gestational Days', 'Birth Weight')
    lines.append([a, b, a * 291 + b, a * 300 + b, a * 309 + b])
    
for i in np.arange(lines.num_rows):
    line = lines.row(i)
    plt.plot([291, 309], [line.item('at 291'), line.item('at 309')], lw=1)
    plt.scatter(300, line.item('at 300'), s=30)

In [ ]:
def bootstrap_prediction(table, x, y, new_x, repetitions=5000):

    # Bootstrap resampling
    predictions = np.empty(repetitions)
    for i in np.arange(repetitions):
        resample = table.sample()
        a = slope(resample, x, y)
        b = intercept(resample, x, y)
        predicted_y = a * new_x + b
        predictions[i] = predicted_y

    # Find the ends of the approximate 95% prediction interval
    left = percentile(2.5, predictions)
    right = percentile(97.5, predictions)

    # Display results
    Table().with_column('Prediction', predictions).hist(bins=20, normed=None, density=True)
    plt.xlabel('predictions at x='+str(new_x))
    plt.plot([left, right], [0, 0], color='yellow', lw=8);
    print('Approximate 95%-confidence interval for height of true line:')
    print(left, right, '(width =', right - left, ')')
    
bootstrap_prediction(baby, 'Gestational Days', 'Birth Weight', 300)

In [ ]:
bootstrap_prediction(baby, 'Gestational Days', 'Birth Weight', 330)

In [ ]:
# lots of data, stricter interval
bootstrap_prediction(baby, 'Gestational Days', 'Birth Weight', 280)

Lec 7.3 The True Slope

In [ ]:
print(slope(baby, 'Gestational Days', 'Birth Weight'))

for i in range(4):
    baby.sample().scatter('Gestational Days', 'Birth Weight', fit_line=True)

In [ ]:
# see slope variation
baby.scatter('Gestational Days', 'Birth Weight')
for i in range(20):
    resample = baby.sample()
    s = slope(resample, 'Gestational Days', 'Birth Weight')
    c = intercept(resample, 'Gestational Days', 'Birth Weight')
    xlims = make_array(150, 350)
    plt.plot(xlims, s*xlims + c, lw=4)

In [ ]:
print(baby.num_rows, baby.sample().num_rows)

counter = 5_000
slopes = np.empty(counter)
for i in range(counter):
    resample = baby.sample()
    resample_slope = slope(resample, 'Gestational Days', 'Birth Weight')
    slopes[i] = resample_slope
Table().with_column('Bootstrap Slopes', slopes).hist(bins=20, normed=None, density=True)

In [ ]:
# reasonable guess for the slope
left = percentile(2.5, slopes)
right = percentile(97.5, slopes)
[left, right]

In [ ]:
def bootstrap_slope(table, x, y, repetitions=5000):
    
    # Bootstrap resampling
    slopes = np.empty(repetitions)
    for i in range(repetitions):
        resample = table.sample()
        resample_slope = slope(resample, x, y)
        slopes[i] = resample_slope
    
    # Find the endpoints of the 95% confidence interval for the true slope
    left = percentile(2.5, slopes)
    right = percentile(97.5, slopes)
    
    # Slope of the regression line from the original sample
    observed_slope = slope(table, x, y)
    
    # Display results
    Table().with_column('Bootstrap Slopes', slopes).hist(bins=20, normed=None, density=True)
    plt.plot([left, right], [0, 0], color='yellow', lw=8);
    print('Slope of regression line:', observed_slope)
    print('Approximate 95%-confidence interval for the true slope:')
    print(left, right)
    
bootstrap_slope(baby, 1, 0)

In [ ]:
plot_residuals(baby, 2, 1)

In [ ]:
# we can't reject a null hypotesis there is no correlation in this case!
bootstrap_slope(baby, 2, 1)